In [2]:
! pip install awswrangler

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import awswrangler as wr
import pandas as pd
import csv
from IPython.display import display, Markdown
import scipy.stats
import numpy as np

In [4]:
pd.options.display.max_columns = None

In [5]:
database_name = 'clarify_23q3'

In [6]:
# query generates allowed cost and PMPM by member for all medical & Rx costs based on the latest 12 months Austin Commercial data from Clarify

query = """
        WITH med_summary as (SELECT patient_id, 
                                    year(claim_start_dt) as inc_year, 
                                    month(claim_start_dt) as inc_month, 
                                    count() as claims, 
                                    sum(charge_allowed_amt)/0.983 as allowed
                             FROM clarify_23q3.claim_header
                             WHERE claim_start_dt BETWEEN DATE('2022-04-01') and DATE('2023-03-31') 
                                   AND claim_status_cd <>'D'
                             GROUP BY 1, 2, 3),

            rx_summary as (SELECT patient_id, 
                                  year(fill_dt) as inc_year, 
                                  month(fill_dt) as inc_month,
                                  count() as claims,
                                  sum(clarify_standardized_amt)*0.81 std_amt
                           FROM clarify_23q3.rx_claim
                           WHERE fill_dt BETWEEN DATE('2022-04-01') and DATE('2023-03-31') 
                           GROUP BY 1, 2, 3)

        SELECT e.patient_id, 
               count(*) mbr_months,
               coalesce(sum(med_summary.allowed),0) med_allowed,
               coalesce(sum(rx_summary.std_amt),0) rx_allowed,
               coalesce(sum(med_summary.allowed),0)+coalesce(sum(rx_summary.std_amt),0) tot_allowed
        FROM clarify_23q3.enrollment e
             LEFT JOIN med_summary ON (med_summary.patient_id = e.patient_id
                                       AND med_summary.inc_year = year_nbr
                                       AND med_summary.inc_month = month_nbr)
             LEFT JOIN rx_summary ON (rx_summary.patient_id = e.patient_id
                                      AND rx_summary.inc_year = year_nbr
                                      AND rx_summary.inc_month = month_nbr)   
             LEFT JOIN garydb.bad_data_23q3 b ON (e.patient_id = b.patient_id
                                                  AND e.year_nbr = b.year_nbr
                                                  AND e.month_nbr = b.month_nbr)
        WHERE coverage_mx_flg = true
              AND coverage_rx_flg = true
              AND e.payer_group_cd = 'C'
              AND e.zip_3 in ('765','786','787','789')
              AND e.first_of_month_enrollment_dt BETWEEN DATE('2022-04-01') and DATE('2023-03-31') 
              AND b.patient_id is null 
        GROUP BY 1
        """




In [7]:
df = wr.athena.read_sql_query(sql=query, database=database_name)

In [8]:
len(df)

425946

In [9]:
import numpy as np

med_trend = 0.07
rx_trend = 0.09
trend_months = 21 # trending from mid (4/1/2022 - 3/31/2023) to (1/1/2024 - 12/31/2024)
df['trended_med'] = (df['med_allowed'].to_numpy(dtype=np.single))*((1+med_trend)**(trend_months/12))
df['trended_rx'] = (df['rx_allowed'].to_numpy(dtype=np.single))*((1+rx_trend)**(trend_months/12))
df['trended_tot'] = df['trended_med'] + df['trended_rx']
df

,patient_id,mbr_months,med_allowed,rx_allowed,tot_allowed,trended_med,trended_rx,trended_tot
0,c3614a3ebad8f4a2b9e18904d673d530,12,2494.872838,7215.4314,9710.304238,2808.471436,8389.936523,11198.408203
1,042bfb6e2ecf6cede6ab1a7f03578ba8,8,1275.361139,0.0000,1275.361139,1435.670532,0.000000,1435.670532
2,d49591f3d29b2e73ea93a8806dd66cec,12,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000
3,fcae20734be23d1a6e2ff00bf5bbafb9,11,1087.670397,5.8968,1093.567197,1224.387573,6.856662,1231.244263
4,6b2694442f63d51d424927a7c764a3f6,12,4342.268566,2.9322,4345.200766,4888.079590,3.409494,4891.489258
...,...,...,...,...,...,...,...,...
425941,6d4f3d7122166adb1111cf8ac44c2177,12,6536.093591,167.2326,6703.326191,7357.662598,194.454193,7552.116699
425942,02c089b958273060d11d71198303f9ca,12,605.991862,0.0000,605.991862,682.163391,0.000000,682.163391
425943,b74229a4bdb2d280cdd20240f9ec1fb0,12,2073.916582,0.0000,2073.916582,2334.602051,0.000000,2334.602051
425944,bb6b9b60b771f14b1ae72787e4802807,12,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000


In [10]:
df[['mbr_months', 'med_allowed','rx_allowed','tot_allowed','trended_med','trended_rx','trended_tot']].sum().map('{:,.2f}'.format)

mbr_months         4,413,411.00
med_allowed    2,240,177,752.08
rx_allowed       502,520,255.04
tot_allowed    2,742,698,007.11
trended_med    2,521,762,048.00
trended_rx       584,318,912.00
trended_tot    3,106,081,024.00
dtype: object

In [11]:
df[['mbr_months', 'med_allowed','rx_allowed','tot_allowed','trended_med','trended_rx','trended_tot']].max().map('{:,.2f}'.format)

mbr_months            12.00
med_allowed    4,437,953.90
rx_allowed     1,182,203.17
tot_allowed    4,441,631.93
trended_med    4,995,792.50
trended_rx     1,374,638.38
trended_tot    5,000,069.00
dtype: object

In [12]:
(df[['med_allowed','rx_allowed','tot_allowed','trended_med','trended_rx','trended_tot']].sum()/df['mbr_months'].sum()).map('{:,.2f}'.format)

med_allowed    507.58
rx_allowed     113.86
tot_allowed    621.45
trended_med    571.39
trended_rx     132.40
trended_tot    703.78
dtype: object

In [13]:
samples = 18
simulations = 125000
LR = 0.65 # future update: LR should be higher for lower ISL deductibles 

# Define the list of plans and savings options
plans = ['A', 'B', 'C', 'D']
savings_options = ['with savings', 'without savings']

# Define the values for coins and oop_max for each plan
plan_values = {
    'A': {'coins': 0.05, 'oop_max': 3000},
    'B': {'coins': 0.1, 'oop_max': 5000},
    'C': {'coins': 0.15, 'oop_max': 7000},
    'D': {'coins': 0.15, 'oop_max': 9000},
}

# Define specific values for each plan and savings option
specific_value_with_savings = {
    'A': 355.39/680.36, 'B': 332.94/659.27, 'C': 312.33/638.34, 'D': 310.12/633.52
}

specific_value_without_savings = {
    'A': 438.21/680.36, 'B': 424.67/659.27, 'C': 412.60/638.34, 'D': 409.67/633.52
}

# Create a function to calculate premiums and save results to CSV
def calculate_and_save_premiums(plan, savings_option, group_size, simulations, LR):
    coins = plan_values[plan]['coins']
    oop_max = plan_values[plan]['oop_max']
    
    # Calculate est_oop and est_paid based on coins and oop_max
    est_oop = np.minimum(df['trended_tot'] * coins, oop_max)
    df['est_oop'] = est_oop
    df['est_paid'] = df['trended_tot'] - est_oop
    
    # Apply adjustment based on plan and savings option
    if savings_option == 'with savings':
        df['est_paid'] = df['est_paid'] * (specific_value_with_savings[plan])
    elif savings_option == 'without savings':
        df['est_paid'] = df['est_paid'] * (specific_value_without_savings[plan])
    
    
    # simulation
    mbr_months = df['mbr_months'].to_numpy(dtype=np.single)
    allowed = df['est_paid'].to_numpy(dtype=np.single) 

    allowed25 = allowed.copy()
    allowed25[allowed25>25000] = 25000

    allowed50 = allowed.copy()
    allowed50[allowed50>50000] = 50000

    allowed75 = allowed.copy()
    allowed75[allowed75>75000] = 75000

    allowed100 = allowed.copy()
    allowed100[allowed100>100000] = 100000

    allowed125 = allowed.copy()
    allowed125[allowed125>125000] = 125000

    allowed150 = allowed.copy()
    allowed150[allowed150>150000] = 150000

    samples_none = []
    samples_25 = []
    samples_50 = []
    samples_75 = [] 
    samples_100 = []
    samples_125 = [] 
    samples_150 = []
    samples_mbr_months = []
    for _ in range(simulations):
        s = np.random.choice(len(df), size = group_size, replace = True)
        samples_mbr_months.append(mbr_months[s].sum())
        samples_none.append(allowed[s].sum()/mbr_months[s].sum())
        samples_25.append(allowed25[s].sum()/mbr_months[s].sum())
        samples_50.append(allowed50[s].sum()/mbr_months[s].sum())
        samples_75.append(allowed75[s].sum()/mbr_months[s].sum())
        samples_100.append(allowed100[s].sum()/mbr_months[s].sum())
        samples_125.append(allowed125[s].sum()/mbr_months[s].sum())
        samples_150.append(allowed150[s].sum()/mbr_months[s].sum())
    
    
    # Calculate premiums 
    samples = [samples_none, samples_150, samples_125, samples_100, samples_75, samples_50, samples_25]

    # Calculate averages for all samples
    averages = [sum(sample) / len(sample) for sample in samples]

    # Calculate percentiles for different average multipliers
    multipliers = [1.1, 1.2, 1.25]
    percentiles = []

    for multiplier in multipliers:
        percentiles.append([scipy.stats.percentileofscore(sample, avg * multiplier) for sample, avg in zip(samples, averages)])

    # Calculate new averages based on minimum values
    new_averages = [
        [sum(min(x, avg * multiplier) for x in sample) / len(sample) for sample, avg in zip(samples, averages)]
        for multiplier in multipliers
    ]

    # Calculate premiums
    premiums = [
        [((x - y) * (1 - z / 100)) / LR for x, y, z in zip(averages, new_avg, percentile)]
        for new_avg, percentile in zip(new_averages, percentiles)
    ]

    # Calculate ISL premiums
    premiums_isl = [(averages[0] - avg) / LR for avg in averages]


    # Save results to CSV
    with open(f'premiums_results_{plan}_{savings_option}.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # Create a list of lists to represent the CSV data
        header = ["Type / Deductible", "None", "$150k Ded.", "$125k Ded.", "$100k Ded.", "$75k Ded." ,"$50k Ded." , "$25k Ded."]
        csv_data = [header] + [["ISL Premiums"] + premiums_isl]
        for i, multiplier in enumerate(multipliers):
            csv_data.append([f"ASL Premium {multiplier}"] + premiums[i])
        
        csvwriter.writerows(csv_data)


# Now, loop through plans and savings options and run the calculation for each
for plan in plans:
    for savings_option in savings_options:
        calculate_and_save_premiums(plan, savings_option, samples, simulations, LR)


In [14]:
# #Convert to Paid 


# coins = 0.05
# oop_max = 3000
# est_oop = np.minimum(df['trended_tot']*coins, oop_max)
# df['est_oop'] = est_oop
# df['est_paid'] = df['trended_tot'] - est_oop
# df

In [15]:
# (df[['est_paid']].sum()/df['mbr_months'].sum()).map('{:,.2f}'.format)

In [16]:
# # Adjustment - scale paid down to equal the specific value for each plan permutation 

# #Plan A - w/ savings 
# df['est_paid'] = df['est_paid']* 454.11/680.36


# # #Plan A - w/o savings 
# # df['est_paid'] = df['est_paid']* 559.93/680.36


# # #Plan B - w/ savings 
# # df['est_paid'] = df['est_paid']* 425.42/680.36


# # #Plan B - w/o savings 
# # df['est_paid'] = df['est_paid']* 542.62/680.36


# # #Plan C - w/ savings 
# # df['est_paid'] = df['est_paid']* 399.10/680.36


# # #Plan C - w/o savings 
# # df['est_paid'] = df['est_paid']* 527.20/680.36

# # #Plan D - w/ savings 
# # df['est_paid'] = df['est_paid']* 396.27/680.36


# # #Plan D - w/o savings 
# # df['est_paid'] = df['est_paid']* 523.46/680.36


In [17]:
# (df[['est_paid']].sum()/df['mbr_months'].sum()).map('{:,.2f}'.format)

## **Simulate Stoploss on 50 Member Group**


In [18]:
# import numpy as np

In [19]:
# mbr_months = df['mbr_months'].to_numpy(dtype=np.single)
# allowed = df['est_paid'].to_numpy(dtype=np.single) 

# allowed25 = allowed.copy()
# allowed25[allowed25>25000] = 25000
# #reimbursement25 = np.maximum(allowed - 25000, 0)

# allowed50 = allowed.copy()
# allowed50[allowed50>50000] = 50000
# #reimbursement50 = np.maximum(allowed - 50000, 0)

# allowed75 = allowed.copy()
# allowed75[allowed75>75000] = 75000
# #reimbursement75 = np.maximum(allowed - 75000, 0)

# allowed100 = allowed.copy()
# allowed100[allowed100>100000] = 100000
# #reimbursement100 = np.maximum(allowed - 100000, 0)

# allowed125 = allowed.copy()
# allowed125[allowed125>125000] = 125000
# #reimbursement125 = np.maximum(allowed - 125000, 0)

# allowed150 = allowed.copy()
# allowed150[allowed150>150000] = 150000
# #reimbursement150 = np.maximum(allowed - 150000, 0)


In [20]:
# # simulate n members over m iterations
# samples_none = []
# samples_25 = []
# samples_50 = []
# samples_75 = [] 
# samples_100 = []
# samples_125 = [] 
# samples_150 = []
# samples_mbr_months = []
# m = 50  # number of members
# n = 50000  # number of simulations to run
# for _ in range(n):
#     s = np.random.choice(len(df), size = m, replace = True)
#     samples_mbr_months.append(mbr_months[s].sum())
#     samples_none.append(allowed[s].sum()/mbr_months[s].sum())
#     samples_25.append(allowed25[s].sum()/mbr_months[s].sum())
#     samples_50.append(allowed50[s].sum()/mbr_months[s].sum())
#     samples_75.append(allowed75[s].sum()/mbr_months[s].sum())
#     samples_100.append(allowed100[s].sum()/mbr_months[s].sum())
#     samples_125.append(allowed125[s].sum()/mbr_months[s].sum())
#     samples_150.append(allowed150[s].sum()/mbr_months[s].sum())

In [21]:
# from matplotlib import pyplot as plt
# from matplotlib.ticker import StrMethodFormatter

# fig, axs = plt.subplots(1, 1)
# axs.hist(samples_none, bins = 500, color='navy')
# axs.xaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
# plt.xlim(0,1200)
# axs.hist(samples_150, bins = 500, color='blue')
# axs.hist(samples_100, bins=500, color='deepskyblue')
# axs.hist(samples_25, bins=500, color='cyan')
# plt.title("{:,}".format(n) + ' simulated samples of ' + "{:,}".format(m) + ' members')
# plt.xlabel('Average paid monthly medical+Rx cost per member')
# plt.ylabel('Frequency of occurrence')
# plt.legend(['No Stoploss','$150K','$100K','$25K'])
# plt.show()

In [22]:
# stats = pd.DataFrame(samples_none).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats.rename(columns={0: "No stoploss"}, inplace=True)
# stats['$150K'] = pd.DataFrame(samples_150).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$125K'] = pd.DataFrame(samples_125).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$100K'] = pd.DataFrame(samples_100).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$75K'] = pd.DataFrame(samples_75).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$50K'] = pd.DataFrame(samples_50).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$25K'] = pd.DataFrame(samples_25).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['Mbr months'] = pd.DataFrame(samples_mbr_months).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['Avg months'] = pd.DataFrame(np.array(samples_mbr_months)/m).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats.drop(['count'], inplace=True)
# for c in stats.columns:
#     stats[c] = stats[c].map('{:,.2f}'.format)
# stats

In [23]:
# # simplified version 

# import scipy.stats
# import csv

# LR = 0.65
# samples = [samples_none, samples_150, samples_125, samples_100, samples_75, samples_50, samples_25]

# # Calculate averages for all samples
# averages = [sum(sample) / len(sample) for sample in samples]

# # Calculate percentiles for different average multipliers
# multipliers = [1.1, 1.2, 1.25]
# percentiles = []

# for multiplier in multipliers:
#     percentiles.append([scipy.stats.percentileofscore(sample, avg * multiplier) for sample, avg in zip(samples, averages)])

# # Calculate new averages based on minimum values
# new_averages = [
#     [sum(min(x, avg * multiplier) for x in sample) / len(sample) for sample, avg in zip(samples, averages)]
#     for multiplier in multipliers
# ]

# # Calculate premiums
# premiums = [
#     [((x - y) * (1 - z / 100)) / LR for x, y, z in zip(averages, new_avg, percentile)]
#     for new_avg, percentile in zip(new_averages, percentiles)
# ]

# # Calculate ISL premiums
# premiums_isl = [averages[0] - avg for avg in averages]

# # Print results
# print("ISL Premiums")
# print(premiums_isl)

# print("ASL Premiums")
# for i, multiplier in enumerate(multipliers):
#     print(f"Multiplier {multiplier}")
#     print(premiums[i])

# # Create a list of lists to represent the CSV data
# csv_data = [["ISL Premiums"] + premiums_isl]
# for i, multiplier in enumerate(multipliers):
#     csv_data.append([f"ASL Premium {multiplier}"] + premiums[i])

# # Write results to a CSV file
# with open('premiums_results.csv', 'w', newline='') as csvfile:
#     csvwriter = csv.writer(csvfile)
#     csvwriter.writerows(csv_data)



In [24]:
# # FIRST PASS 

# import scipy.stats

# LR = 0.65
# samples = [samples_none, samples_150, samples_125, samples_100, samples_75, samples_50, samples_25]
# samples_agg1 = []
# samples_agg2 = []
# samples_agg3 = [] 
# averages = [] 
# percentiles_agg1 = [] 
# percentiles_agg2 = [] 
# percentiles_agg3 = [] 
# new_averages_agg1 = []
# new_averages_agg2 = [] 
# new_averages_agg3 = [] 
# premiums_agg1 = [] 
# premiums_agg2 = [] 
# premiums_agg3 = [] 

# for sample in samples:
#     average = sum(sample) / len(sample)
#     averages.append(average)
    
    
# averages_agg1 = [value * 1.1 for value in averages]
# averages_agg2 = [value * 1.2 for value in averages]
# averages_agg3 = [value * 1.25 for value in averages]

# for i, sample in enumerate(samples):
#     percentiles_agg1.append(scipy.stats.percentileofscore(sample, averages_agg1[i]))

# for i, sample in enumerate(samples):
#     percentiles_agg2.append(scipy.stats.percentileofscore(sample, averages_agg2[i]))

# for i, sample in enumerate(samples):
#     percentiles_agg3.append(scipy.stats.percentileofscore(sample, averages_agg3[i]))   

    

    
    
# for i, sample in enumerate(samples):
#     agg = [min(x, averages_agg1[i]) for x in sample] 
#     samples_agg1.append(agg) 
    

# for i, sample in enumerate(samples):
#     agg = [min(x, averages_agg2[i]) for x in sample] 
#     samples_agg2.append(agg) 
    

# for i, sample in enumerate(samples):
#     agg = [min(x, averages_agg3[i]) for x in sample] 
#     samples_agg3.append(agg) 
    
# for sample in samples_agg1:
#     average = sum(sample) / len(sample)
#     new_averages_agg1.append(average)    
    
# for sample in samples_agg2:
#     average = sum(sample) / len(sample)
#     new_averages_agg2.append(average)  

# for sample in samples_agg3:
#     average = sum(sample) / len(sample)
#     new_averages_agg3.append(average)  

# premiums_agg1 = [((x - y) * (1-z/100))/LR for x, y, z in zip(averages, new_averages_agg1, percentiles_agg1)]
# premiums_agg2 = [((x - y) * (1-z/100))/LR for x, y, z in zip(averages, new_averages_agg2, percentiles_agg2)]
# premiums_agg3 = [((x - y) * (1-z/100))/LR for x, y, z in zip(averages, new_averages_agg3, percentiles_agg3)]
    

# premiums_isl = [averages[0] - average for average in averages]

# total1 = [x + y for x, y in zip(premiums_isl, premiums_agg1)]
# total2 = [x + y for x, y in zip(premiums_isl, premiums_agg2)]
# total3 = [x + y for x, y in zip(premiums_isl, premiums_agg3)]

# print("ISL premiums")     
# print(premiums_isl)
    
# print("ASL premiums")
# print(premiums_agg1)
# print(premiums_agg2)
# print(premiums_agg3)

# print("total premiums")
# print(total1)
# print(total2)
# print(total3)


In [25]:
# #simple code to check indices 

# print(i)
# print(LR)
# print(averages[i])
# print(averages_agg3[i])
# print(percentiles_agg3[i])
# print(new_averages_agg3[i])
# print(((averages[i] - new_averages_agg3[i])*(1-percentiles_agg3[i]/100))/LR)

In [26]:

# #Simple code to test my logic 

# LR = 0.65

# i = 6

# agg = 1.25

# sample = samples[i]

# average = sum(sample) / len(sample)

# attachment =  average*agg 

# percentile = scipy.stats.percentileofscore(sample, attachment)

# sample_agg = [min(x, attachment) for x in sample] 

# new_average_agg = sum(sample_agg) / len(sample_agg)

# premium = ((average - new_average_agg) * (1 - percentile/100))/LR 

# print(f"Average: {average}")
# print(f"Attachment: {attachment}")
# print(f"Percentile: {percentile}")
# print(f"New Average: {new_average_agg}")
# print(f"Premium: {premium}")


In [27]:
# shape_fit, loc_fit, scale_fit = scipy.stats.lognorm.fit(samples_none_above_without_zeros)

# print(shape_fit)

# print(loc_fit)

# print(scale_fit)


Distribution of Large Claims 

In [28]:
# # simulate n members over m iterations
# samples_25 = []
# samples_50 = []
# samples_75 = [] 
# samples_100 = []
# samples_125 = [] 
# samples_150 = []
# samples_mbr_months = []
# m = 50  # number of members
# n = 1000000  # number of simulations to run
# for _ in range(n):
#     s = np.random.choice(len(df), size = m, replace = True)
#     samples_mbr_months.append(mbr_months[s].sum())
#     samples_25.append(reimbursement25[s].sum()/mbr_months[s].sum())
#     samples_50.append(reimbursement50[s].sum()/mbr_months[s].sum())
#     samples_75.append(reimbursement75[s].sum()/mbr_months[s].sum())
#     samples_100.append(reimbursement100[s].sum()/mbr_months[s].sum())
#     samples_125.append(reimbursement125[s].sum()/mbr_months[s].sum())
#     samples_150.append(reimbursement150[s].sum()/mbr_months[s].sum())

In [29]:
# fig, axs = plt.subplots(1, 1)
# axs.hist(samples_150, bins = 500, color='navy')
# axs.xaxis.set_major_formatter(StrMethodFormatter('${x:,.0f}'))
# plt.xlim(200,1200)
# axs.hist(samples_100, bins=500, color='deepskyblue')
# axs.hist(samples_25, bins=500, color='cyan')
# plt.title("{:,}".format(n) + ' simulated samples of ' + "{:,}".format(m) + ' members')
# plt.xlabel('Average paid monthly medical+Rx cost per member')
# plt.ylabel('Frequency of occurrence')
# plt.legend(['$150K','$100K','$25K'])
# plt.show()

In [30]:
# stats = pd.DataFrame(samples_150).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats.rename(columns={0: "150k"}, inplace=True)
# stats['$125K'] = pd.DataFrame(samples_125).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$100K'] = pd.DataFrame(samples_100).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$75K'] = pd.DataFrame(samples_75).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$50K'] = pd.DataFrame(samples_50).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['$25K'] = pd.DataFrame(samples_25).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['Mbr months'] = pd.DataFrame(samples_mbr_months).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats['Avg months'] = pd.DataFrame(np.array(samples_mbr_months)/m).describe(percentiles = [0.01, 0.05, 0.50, 0.95, 0.99])
# stats.drop(['count'], inplace=True)
# for c in stats.columns:
#     stats[c] = stats[c].map('{:,.2f}'.format)
# stats

Fit data to a Pareto 

In [31]:
# import scipy.stats




# dep = allowed.tolist() 

# shape_fit, loc_fit, scale_fit = scipy.stats.gamma.fit(dep)

# print(shape_fit)

# print(loc_fit)

# print(scale_fit)




In [32]:

# x = np.linspace(np.min(dep), np.max(dep)) 

# plt.plot(x, scipy.stats.gamma.pdf(x, shape_fit, loc_fit, scale_fit), ) 
# plt.xlabel("paid amount") 
# plt.ylabel("Probability Density")

In [33]:
# plt.show() 

In [34]:
# # Threshold value 'a'
# a = 1.1*454.19  # Replace with your desired threshold

# # Calculate the conditional expectation
# def conditional_expectation(x):
#     pdf = scipy.stats.lognorm.pdf(x, shape_fit, loc=loc_fit, scale=scale_fit)
#     cdf_gt_a = 1 - scipy.stats.lognorm.cdf(a, shape_fit, loc=loc_fit, scale=scale_fit)
#     return x * pdf / cdf_gt_a

# conditional_expectation_result, _ = scipy.integrate.quad(conditional_expectation, a, scipy.inf)
# print("Conditional Expectation:", conditional_expectation_result)

In [35]:
# print(1 - scipy.stats.lognorm.cdf(a, shape_fit, loc=loc_fit, scale=scale_fit))

In [36]:
# 0.505*8317

In [37]:
# 4200/0.65